## Dataset details and links ##

The sample dataset is taken from the Tableau public webiste where there is a beautiful rendition of how the variables are to be viewed in the form of a dashboard. 
However the df_extra dataset is a random dataset which is synthetically generated. The source cannot be disclosed in this particular notebook.
- The link to the dashboard is here : https://www.tableau.com/data-insights/dashboard-showcase/superstore
- The link to the datasets are : https://drive.google.com/drive/folders/1Jm5hRf0OFIC9X10CKwmuUHpR0gwCHRHT?usp=share_link

This notebook attempts to discover secret patterns in the dataset using a lot of statistical measures and indepedent graphical representations of the data. 

**Details on how the notebook is to be viewed and handled**
- The custom made functions that are used in the notebook are imported from the functions.py file in the same directory so that the main goal of the project is not being deviated.
- This particular notebook contains all the information that not plagiarised from any sources. This is a completely independent work and people are open to take references from this notebook.
- The datasets used for this particular holds no connection with any personal information and all of this is completely available in the internet for personal usage.
- This notebook is to be used exclusively for education purposes.

**Importing the required libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import warnings
warnings.filterwarnings('ignore')
import datetime
import math
from functions import col_names

In [2]:
#importing the required datasets
df_orders = pd.read_excel(r"C:\Users\subha\Downloads\sample_-_superstore.xlsx", sheet_name= 'Orders')
df_returns = pd.read_excel(r"C:\Users\subha\Downloads\sample_-_superstore.xlsx", sheet_name = "Returns")
df_people = pd.read_excel(r"C:\Users\subha\Downloads\sample_-_superstore.xlsx", sheet_name = "People")
df_extra = pd.read_csv(r"C:\Users\subha\Downloads\Superstore event data.xlsx - Sheet1.csv")

print(df_orders.shape, df_returns.shape, df_people.shape, df_extra.shape)

(9994, 21) (296, 2) (4, 2) (136, 4)


In [3]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [4]:
df_returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Returned  296 non-null    object
 1   Order ID  296 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB


In [5]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Person  4 non-null      object
 1   Region  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [6]:
df_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Promotion Date  136 non-null    object
 1   Manufacturer    136 non-null    object
 2   Category        136 non-null    object
 3   Promotion Type  136 non-null    object
dtypes: object(4)
memory usage: 4.4+ KB


In [7]:
#formatting the column names for each and every dataframes
dataframes = [df_orders, df_returns, df_people, df_extra]
for i in dataframes:
    col_names(i)
print(df_orders.columns, df_returns.columns, df_people.columns, df_extra.columns)

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales', 'quantity', 'discount', 'profit'],
      dtype='object') Index(['returned', 'order_id'], dtype='object') Index(['person', 'region'], dtype='object') Index(['promotion_date', 'manufacturer', 'category', 'promotion_type'], dtype='object')


In [8]:
#formatting the datapoints in the dataframes and adding the columns that are required
df_orders['cost_price'] = df_orders['sales'] - df_orders['profit']
df_orders['selling_price'] = df_orders['cost_price'] / (1- df_orders['discount'])
df_orders['total_revenue'] = df_orders['selling_price'] * df_orders['quantity']
df_orders['gross_profit'] = df_orders['total_revenue'] - df_orders['cost_price']
df_orders['gross_margin'] = (df_orders['gross_profit'] / df_orders['total_revenue'])*100
df_orders.head(3)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,product_name,sales,quantity,discount,profit,cost_price,selling_price,total_revenue,gross_profit,gross_margin
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,220.0464,220.0464,440.0928,220.0464,50.000000
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820,512.3580,512.3580,1537.0740,1024.7160,66.666667
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714,7.7486,7.7486,15.4972,7.7486,50.000000


**Additional Parameters that can be derived from the given data**

- Gross profit: Gross profit is the difference between revenue and cost of goods sold (COGS). It can be calculated by subtracting the COGS from the revenue. The formula for gross profit is: Gross Profit = Revenue - COGS.

- Gross margin: Gross margin is the percentage of revenue that exceeds COGS. It can be calculated by dividing gross profit by revenue and multiplying the result by 100. The formula for gross margin is: Gross Margin = (Gross Profit / Revenue) * 100.

- Average order value: Average order value is the average amount of revenue generated per order. It can be calculated by dividing the total revenue by the number of orders. The formula for average order value is: Average Order Value = Total Revenue / Number of Orders.

- Sales growth: Sales growth is the percentage increase or decrease in revenue over a specific period of time. It can be calculated by subtracting the previous period's revenue from the current period's revenue, dividing the result by the previous period's revenue, and multiplying the result by 100. The formula for sales growth is: Sales Growth = ((Current Period Revenue - Previous Period Revenue) / Previous Period Revenue) * 100.

- Customer lifetime value: Customer lifetime value is the total revenue generated by a customer over their entire relationship with the business. It can be calculated by multiplying the average order value by the number of orders per customer and the average customer lifespan. The formula for customer lifetime value is: Customer Lifetime Value = Average Order Value * Number of Orders per Customer * Average Customer Lifespan.

- Profit Margin: Profit margin is the ratio of profit to revenue. It measures the profitability of a company's products or services. It can be calculated as (profit / revenue) x 100.

- Discount Percentage: Discount percentage is the percentage of discount given on each product. It can be calculated as (discount / selling price) x 100.

- Sales Growth: Sales growth is the percentage change in revenue over a period of time. It can be calculated as ((current period revenue - previous period revenue) / previous period revenue) x 100.

- Customer Lifetime Value: Customer lifetime value is the amount of revenue a customer is expected to generate over the course of their relationship with a company. It can be calculated as (average revenue per customer x average customer lifespan).

In [9]:
df_orders.head(3)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,product_name,sales,quantity,discount,profit,cost_price,selling_price,total_revenue,gross_profit,gross_margin
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,220.0464,220.0464,440.0928,220.0464,50.000000
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820,512.3580,512.3580,1537.0740,1024.7160,66.666667
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714,7.7486,7.7486,15.4972,7.7486,50.000000


In [11]:
df = pd.merge(df_orders, df_returns , how = "left", on= 'order_id')
df.drop_duplicates(inplace = True)
df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,sales,quantity,discount,profit,cost_price,selling_price,total_revenue,gross_profit,gross_margin,returned
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,261.9600,2,0.00,41.9136,220.0464,220.0464,440.0928,220.0464,50.000000,NaN
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,731.9400,3,0.00,219.5820,512.3580,512.3580,1537.0740,1024.7160,66.666667,NaN
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,14.6200,2,0.00,6.8714,7.7486,7.7486,15.4972,7.7486,50.000000,NaN
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,957.5775,5,0.45,-383.0310,1340.6085,2437.4700,12187.3500,10846.7415,89.000000,NaN
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,22.3680,2,0.20,2.5164,19.8516,24.8145,49.6290,29.7774,60.000000,NaN


In [13]:
#filling up the missing values which is only present in the returned column with "No" values
df.returned.fillna("No", inplace = True)